<a href="https://colab.research.google.com/github/pepborrell/LauzHack2019/blob/master/LauzHack2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install client library
!pip3 install --upgrade google-cloud-vision

In [0]:
!export GOOGLE_APPLICATION_CREDENTIALS="./LauzHack2019-81776abcfeb4.json"

In [0]:
!mkdir meme_json

##Neteja de JSONS:

In [0]:
import json
from collections import namedtuple 
import pandas as pd
MemeData = namedtuple('MemeData', ["upvotes","annotations", "text",   "matchparcial",
                                    "matchpages", "matchsimilar", "webLabels"])
def processGoogleResult(meme, upvotes):
  # Annotations
  Annotation = namedtuple('Annotation', ['name', 'score'])
  annotations = []
  for ann in meme.get("labelAnnotations",[]):
    annotations.append(Annotation(ann["description"],ann["score"]))

  # meme text
  text = meme.get("fullTextAnnotation",{}).get("text","")

  # Web entities
  webthings = meme.get("webDetection",{})
  for ent in webthings.get("webEntities",[]):
    try:
      annotations.append(Annotation(ent["description"],ent["score"]))
    except:
      pass
  matchparcial = len(webthings.get("partialMatchingImages",[]))
  matchpages = len(webthings.get("pagesWithMatchingImages",[]))
  matchsimilar = len(webthings.get("visuallySimilarImages",[]))
  webLabels = []
  for label in webthings["bestGuessLabels"]:
    try:
      webLabels.append(label["label"])
    except:
      pass
  return MemeData(upvotes, annotations, text,  matchparcial, matchpages,
                matchsimilar, webLabels)

##Crides a api de google

In [91]:
from google.protobuf.json_format import MessageToJson
from google.cloud import vision
from google.cloud.vision import enums
from google.cloud.vision import types
import threading
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="./LauzHack2019-81776abcfeb4.json"
def detect_data(in_path):
    """Detects labels in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    features = [
      types.Feature(type=enums.Feature.Type.TEXT_DETECTION),
      types.Feature(type=enums.Feature.Type.LABEL_DETECTION),
      types.Feature(type=enums.Feature.Type.WEB_DETECTION),
    ]
    if os.stat(in_path).st_size > 1024*1024*4:
      return -1
    with io.open(in_path, 'rb') as image_file:

        content = image_file.read()

    image = vision.types.Image(content=content)

    req = types.AnnotateImageRequest(image=image, features=features)
    response = client.annotate_image(req)
    upvotes = int(in_path.split("/")[-1].split("___")[0])
    #response = client.label_detection(image=image)
    return processGoogleResult(json.loads(MessageToJson(response)), upvotes)


from os import listdir
from os.path import isfile, join

in_path = "./memes/"
out_path = "./meme_json/"

onlyfiles = [f for f in listdir(in_path) if isfile(join(in_path, f))]


N_THREADS = 20
MAX_MEMES = 1000
memes_tmp = [[]for x in range(N_THREADS)]

def work_bitch(segment):
  start = int(segment* (MAX_MEMES/N_THREADS))
  end = start + int(MAX_MEMES/N_THREADS)
  count = start
  for f in onlyfiles[start:end]:
    print("Generating json: {}".format(count))
    resdata = detect_data(join(in_path, f))
    if resdata == -1:
      print("Too big")
      continue
    memes_tmp[segment].append(resdata)
    count += 1

threads = []
for tn in range(N_THREADS):
  x = threading.Thread(target=work_bitch,args=(tn,))
  x.start()
  threads.append(x)
for thread in threads:
  thread.join()

memes = []
for tn in range(N_THREADS):
  memes += memes_tmp[tn]
print("Finished")

Generating json: 0
Generating json: 50Generating json: 100

Generating json: 200
Generating json: 150
Generating json: 250
Generating json: 300
Generating json: 400Generating json: 350Generating json: 450


Generating json: 500Generating json: 550

Generating json: 600
Generating json: 650
Generating json: 700
Generating json: 750
Generating json: 800Generating json: 850Generating json: 900


Generating json: 950
Generating json: 51
Generating json: 1
Generating json: 151
Too big
Generating json: 151
Generating json: 451Generating json: 801

Generating json: 201
Generating json: 551
Generating json: 601
Generating json: 701
Generating json: 851Generating json: 351
Generating json: 951

Generating json: 251Generating json: 651

Generating json: 301
Generating json: 101
Too big
Generating json: 101
Generating json: 401
Generating json: 751
Generating json: 901Too big

Generating json: 751
Generating json: 501
Too big
Generating json: 501
Generating json: 452
Generating json: 552
Generati

In [105]:
memes2 = []
for meme in memes:
  newanon = [[a.name,a.score] for a in meme.annotations]
  memes2.append(MemeData(meme.upvotes, newanon, meme.text,  meme.matchparcial, meme.matchpages,
                meme.matchsimilar, meme.webLabels))
df = pd.DataFrame.from_records(
   memes2,
   columns=MemeData._fields
)
print("Generating...")
df.to_csv("memes.csv")
df.head()

Generating...


,upvotes,annotations,text,matchparcial,matchpages,matchsimilar,webLabels
0,128,"[[Skin, 0.9325852990150452], [Shoulder, 0.9041...","Girl: So, what do you do as a career?\nMe: I'm...",8,10,10,[dark stock]
1,21,"[[Photo caption, 0.7912240624427795], [Font, 0...",",000,000 TREES!\nNew York news\n03:41 PM\nBREA...",10,10,8,[mr beast planting 20 million trees]
2,38,"[[Friendship, 0.8710828423500061], [Forehead, ...",Me watching\nanimal shows\nwith my cat\nMy cat...,10,10,10,[michelle obama young]
3,23,"[[Font, 0.991735577583313], [Text, 0.989824891...","If you are looking for\nan image, it was\nprob...",0,0,10,[adoro la paz]
4,29,"[[Cartoon, 0.9145615696907043], [Arm, 0.859736...",people who refrain\nfrom watching porn\nduring...,10,10,10,[buff mickey mouse meme]


In [0]:
# read urls from csv

import pandas as pd

url = 'https://pastebin.com/raw/SjMqw5Wx'
df = pd.read_csv(url)



In [87]:
# download memes with url list from df to local folder

import requests

ii = 0

for row in df.itertuples():
  upvotes = row.upvotes
  imageUrl = row.url
  fname = imageUrl.split("/")[-1]
  fname2 = str(upvotes) + "___" + fname
  print(fname, fname2)
  try:
    os.rename("./memes/" +fname , "./memes/" +fname2)
  except:
    print("Err")
  print(ii)
  ii +=1


zhh7urcrtyv31.png 39___zhh7urcrtyv31.png
0
lbanftpptyv31.jpg 1___lbanftpptyv31.jpg
1
jpfds36rtyv31.jpg 21___jpfds36rtyv31.jpg
2
hrcbzuvjtyv31.png 30___hrcbzuvjtyv31.png
3
rb8pod5otyv31.png 31___rb8pod5otyv31.png
4
9uty1taptyv31.png 24___9uty1taptyv31.png
5
kqpnfcvmtyv31.png 11___kqpnfcvmtyv31.png
6
dvvt2pwntyv31.png 16___dvvt2pwntyv31.png
7
ykumgrbotyv31.png 1___ykumgrbotyv31.png
8
8686vt3otyv31.png 2___8686vt3otyv31.png
9
2rgzkfvityv31.png 21___2rgzkfvityv31.png
10
ObCeLg7.png 8___ObCeLg7.png
11
ift3jqeltyv31.jpg 24___ift3jqeltyv31.jpg
12
x7qy7scltyv31.jpg 1___x7qy7scltyv31.jpg
13
ayqkvdvhtyv31.jpg 26___ayqkvdvhtyv31.jpg
14
i6reuc2ityv31.jpg 37___i6reuc2ityv31.jpg
15
uwwuzxbgtyv31.jpg 20___uwwuzxbgtyv31.jpg
16
ygigyheftyv31.png 13___ygigyheftyv31.png
17
jwtk4x8ftyv31.jpg 1___jwtk4x8ftyv31.jpg
18
bh3j0inetyv31.jpg 36___bh3j0inetyv31.jpg
19
5mlcxzjetyv31.jpg 31___5mlcxzjetyv31.jpg
20
44beo6jdtyv31.png 22___44beo6jdtyv31.png
21
ralxcdo7tyv31.png 17___ralxcdo7tyv31.png
22
KLnCMIQ.jpg 9___

In [0]:
import time
ii = 0

while True:
  print(ii)
  time.sleep(5)
  ii += 1

    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27